In [84]:
%pip install bs4 requests


Note: you may need to restart the kernel to use updated packages.


In [85]:
from bs4 import BeautifulSoup
import requests
import re
import os
from collections import deque
import json

In [86]:
headers = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/128.0.0.0 Safari/537.36"
    )
}

In [87]:
def crawl_singer_award(soup):
    # Sử dụng set() để tự động chống trùng lặp là chính xác!
    awards = set()
    tables = soup.find_all("table", class_="wikitable")

    for table in tables:
        for row in table.find_all("tr"):
            cells = row.find_all(["td", "th"])
            if len(cells) < 2:
                continue  # Bỏ qua hàng không có đủ ô

            award_cell = cells[1]

            # Logic gốc của bạn: chỉ lấy những ô có thẻ <sup>
            if award_cell.find("sup"):
                text_parts = []
                # Lặp qua các nội dung con (gồm text và tag)
                for content in award_cell.contents:
                    if content.name == "sup":
                        break  # Dừng lại khi gặp <sup> đầu tiên
                    
                    # Lấy text, dù nó là chuỗi trần hay nằm trong tag khác (như <a>, <b>)
                    if isinstance(content, str):
                        text_parts.append(content)
                    elif hasattr(content, "get_text"):
                        text_parts.append(content.get_text())

                # 1. Ghép tất cả các phần text lại
                text = "".join(text_parts)
                
                # 2. Chuẩn hóa khoảng trắng:
                #    Biến "Giải thưởng\n   Ca sĩ" thành "Giải thưởng Ca sĩ"
                text = " ".join(text.split())

                if text:
                    awards.add(text)  # set sẽ tự lo việc chống trùng lặp

    return list(awards)


In [88]:
link_to_check = [
    "https://vi.wikipedia.org/wiki/Soobin_Ho%C3%A0ng_S%C6%A1n",
    "https://vi.wikipedia.org/wiki/H%C3%B2a_Minzy",
    "https://vi.wikipedia.org/wiki/Rhymastic",
    "https://vi.wikipedia.org/wiki/%C4%90en_(rapper)"
]

In [89]:
for link in link_to_check:
    soup = BeautifulSoup(requests.get(link, headers=headers).text, 'html.parser')
    # print(soup)
    print(f"Awards for {link.split('/')[-1]}: {crawl_singer_award(soup)}")
    

Awards for Soobin_Ho%C3%A0ng_S%C6%A1n: ['Male Icon Awards', 'Làn Sóng Xanh', 'Spotify', 'Keeng Young Awards', 'Elle Style Awards', 'Giải thưởng Cống hiến', 'WeChoice Awards', 'Zing Music Awards']
Awards for H%C3%B2a_Minzy: ['Vietnam iContent Awards', 'Vietnam Entertaiment Awards', 'Giải thưởng Cống hiến', 'Làn sóng xanh', 'TikTok Awards Việt Nam']
Awards for Rhymastic: ['Giải thưởng Âm nhạc Cống hiến']


KeyboardInterrupt: 